In [738]:
import pandas as pd
from etna.datasets import TSDataset
from etna.transforms import LagTransform, DateFlagsTransform
from etna.models import CatBoostPerSegmentModel
from etna.pipeline import Pipeline
from etna.metrics import RMSE


In [739]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from statsmodels.tsa.seasonal import STL
import seaborn as sns
from scipy import stats
import itertools
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.tsa.stattools import acf
from statsmodels.tsa.stattools import adfuller
from sklearn.model_selection import GridSearchCV, KFold
from catboost import CatBoostRegressor
from etna.models import CatBoostPerSegmentModel
from etna.metrics import RMSE
from etna.transforms import LagTransform, MeanTransform, DateFlagsTransform
from etna.pipeline import Pipeline
from etna.datasets import TSDataset
from sklearn.model_selection import train_test_split

In [740]:
data_path = './hackathon/data1.csv'
data = pd.read_csv(data_path)

# Преобразуем столбцы и создаем сегменты
data['segment'] = 'segment_1'
data = data.rename(columns={"date": "timestamp", "value": "target"})
data['timestamp'] = pd.to_datetime(data['timestamp'])


In [741]:
# from etna.transforms import LagTransform
# lag_features = LagTransform(in_column="target", out_column="lags", lags=[1, 2, 3])

# ts_with_features = TSDataset(data, freq="D")
# ts_with_features.fit_transform([lag_features])
# ts_with_features = ts_with_features.df["segment_1"]['lags_1'] = ts_with_features.df["segment_1"]['lags_1'].fillna(0)
# ts_with_features


In [742]:
from etna.transforms import MeanTransform
mean_tr = MeanTransform(in_column="target", out_column="mean", window=5)
ts_with_features = TSDataset(data, freq="D")

In [743]:
# from etna.transforms import DateFlagsTransform
# date_tr = DateFlagsTransform(out_column="date_flags", 
#                              day_number_in_week=True,
#                              day_number_in_month=True,
#                              week_number_in_month=True,
#                              is_weekend=True,
#                             )
# date_flag_ts = TSDataset(data, freq="D")
# ts_with_features.fit_transform([date_tr])


In [744]:
import pandas as pd
from sklearn.model_selection import train_test_split
from etna.datasets import TSDataset

# Предположим, что ts_with_features — это ваш TSDataset
# Для начала преобразуем TSDataset в pandas DataFrame
df = ts_with_features.df # df — это DataFrame внутри TSDataset

# Сортируем данные по временной метке
df = df.sort_values(by="timestamp")

# Разделяем данные на обучающую и тестовую выборки (80% train, 20% test)
train_df, test_df = train_test_split(df, test_size=0.2, shuffle=False)

# Разделяем тестовую выборку на валидационную (50% valid, 50% test)
valid_df, test_df = train_test_split(test_df, test_size=0.5, shuffle=False)

# Преобразуем обратно в TSDataset
train_tsd = TSDataset(df=train_df, freq="D")
valid_tsd = TSDataset(df=valid_df, freq="D")
test_tsd = TSDataset(df=test_df, freq="D")



In [745]:
# MeanTransform(in_column="target", out_column="mean", window=5), 
#                                                     
# DateFlagsTransform(in_column="target", out_column="date_flags", 
#                              day_number_in_week=True,
#                              day_number_in_month=True,
#                              week_number_in_month=True,
#                              is_weekend=True,)

In [746]:
train_tsd.columns

MultiIndex([('segment_1', 'target')],
           names=['segment', 'feature'])

In [747]:
train_tsd

segment,segment_1
feature,target
timestamp,
2022-01-01,898.0
2022-01-02,3167.0
2022-01-03,3380.0
2022-01-04,3423.0
2022-01-05,3373.0
...,...
2024-02-14,2486.0
2024-02-15,1801.0


In [748]:
valid_tsd

segment,segment_1
feature,target
timestamp,
2024-02-19,2106.0
2024-02-20,2343.0
2024-02-21,2746.0
2024-02-22,3790.0
2024-02-23,4696.0
...,...
2024-05-21,2343.0
2024-05-22,2294.0


In [749]:
from etna.transforms import DateFlagsTransform, HolidayTransform

date_tr = DateFlagsTransform(out_column="date_flags", 
                             day_number_in_week=True,
                             day_number_in_month=True,
                             week_number_in_month=True,
                             is_weekend=True,
                            )

In [750]:
from etna.transforms import LagTransform
lag_features = LagTransform(in_column="target", out_column="lags", lags=[1, 2, 3])

In [751]:
holiday = HolidayTransform(iso_code='RUS', out_column='holidays')

In [752]:
from etna.transforms import (
    LinearTrendTransform,
    DensityOutliersTransform,
    TimeSeriesImputerTransform,
    StandardScalerTransform
)

transforms = [
    DensityOutliersTransform(
        in_column="target",
        window_size=45,
        n_neighbors=25,
        distance_coef=1.9
    ),
    TimeSeriesImputerTransform(
        in_column="target",
        strategy="running_mean"
    ),
    LinearTrendTransform(in_column="target"),
    LagTransform(in_column="target", out_column="lags", lags=[1, 2, 3]),
    DateFlagsTransform(out_column="date_flags", 
                             day_number_in_week=True,
                             day_number_in_month=True,
                             week_number_in_month=True,
                             is_weekend=True,
                            ),
    MeanTransform(in_column="target", out_column="mean", window=5),
    StandardScalerTransform(in_column='target', out_column='target')
]

/home/denis/miniforge3/envs/hackathon_time/lib/python3.10/site-packages/etna/transforms/math/sklearn.py:72: UserWarning: Transformation will be applied inplace, out_column param will be ignored
  warnings.warn("Transformation will be applied inplace, out_column param will be ignored")


TypeError: TreeFeatureSelectionTransform.__init__() missing 1 required positional argument: 'model'

In [735]:
from etna.models import CatBoostPerSegmentModel
from etna.transforms import LagTransform, MeanTransform, DateFlagsTransform
from etna.pipeline import Pipeline
from etna.metrics import RMSE
import numpy as np
import matplotlib.pyplot as plt

# 1. Модель CatBoost с исправленными параметрами
catboost_model_2 = CatBoostPerSegmentModel(
    n_estimators=250,  # количество деревьев
    learning_rate=0.1,  # скорость обучения
    depth=6,  # глубина деревьев
    random_state=42  # фиксируем random_state для воспроизводимости
)

pipe = Pipeline(model=catboost_model_2, transforms=transforms, 
                                                    horizon=30)

metrics, forecasts, _ = pipe.backtest(train_tsd, metrics=[RMSE()], aggregate_metrics=True, n_folds=5)



[Parallel(n_jobs=1)]: Done   1 tasks      | elapsed:    0.4s
[Parallel(n_jobs=1)]: Done   2 tasks      | elapsed:    0.7s
[Parallel(n_jobs=1)]: Done   3 tasks      | elapsed:    1.1s
[Parallel(n_jobs=1)]: Done   4 tasks      | elapsed:    1.4s
[Parallel(n_jobs=1)]: Done   5 tasks      | elapsed:    1.7s
[Parallel(n_jobs=1)]: Done   5 tasks      | elapsed:    1.7s
[Parallel(n_jobs=1)]: Done   1 tasks      | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done   2 tasks      | elapsed:    0.2s
[Parallel(n_jobs=1)]: Done   3 tasks      | elapsed:    0.3s
[Parallel(n_jobs=1)]: Done   4 tasks      | elapsed:    0.4s
[Parallel(n_jobs=1)]: Done   5 tasks      | elapsed:    0.5s
[Parallel(n_jobs=1)]: Done   5 tasks      | elapsed:    0.5s
[Parallel(n_jobs=1)]: Done   1 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done   2 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done   3 tasks      | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done   4 tasks      | elapsed:    0.1s
[Parallel(n_jobs=1)]: Do

In [736]:
metrics.mean

<bound method NDFrame._add_numeric_operations.<locals>.mean of      segment        RMSE
0  segment_1  911.461448>

In [737]:
rmse = RMSE()
rmse_2 = rmse(y_true=valid_tsd, y_pred=forecast_ts_2)
print(f"Raw RMSE: {rmse_2['raw']}")

# 8. Выводим RMSE
rmse_value = (rmse_2['raw'])
print(f"RMSE: {round(rmse_value, 2)}")

ValueError: y_true and y_pred have different timestamps

In [229]:
model = CatBoostPerSegmentModel()

pipe = Pipeline(model=model, transforms=[lag_3_days, mean_tr, date_tr], 
                horizon=30)

rmse = RMSE()



metrics, forecast, info = pipe.backtest(ts=ts_with_features, metrics=[rmse], n_folds=3)


[Parallel(n_jobs=1)]: Done   1 tasks      | elapsed:    0.8s
[Parallel(n_jobs=1)]: Done   2 tasks      | elapsed:    1.8s
[Parallel(n_jobs=1)]: Done   3 tasks      | elapsed:    2.7s
[Parallel(n_jobs=1)]: Done   3 tasks      | elapsed:    2.7s
[Parallel(n_jobs=1)]: Done   1 tasks      | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done   2 tasks      | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done   3 tasks      | elapsed:    0.2s
[Parallel(n_jobs=1)]: Done   3 tasks      | elapsed:    0.2s
[Parallel(n_jobs=1)]: Done   1 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done   2 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done   3 tasks      | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done   3 tasks      | elapsed:    0.1s


In [230]:
metrics.mean()

/tmp/ipykernel_404484/3493762479.py:1: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  metrics.mean()


RMSE           675.476023
fold_number      1.000000
dtype: float64

In [152]:
ts_with_features.head()

segment                         segment_1                                \
feature    date_flags_day_number_in_month date_flags_day_number_in_week   
timestamp                                                                 
2022-01-01                              1                             5   
2022-01-02                              2                             6   
2022-01-03                              3                             0   
2022-01-04                              4                             1   
2022-01-05                              5                             2   

segment                                                                        \
feature    date_flags_is_weekend date_flags_week_number_in_month         mean   
timestamp                                                                       
2022-01-01                  True                               1   898.000000   
2022-01-02                  True                               1  2032.500000   
2022-01-03                 False                               2  2481.666667   
2022-01-04                 False                               2  2717.000000   
2022-01-05                 False                               2  2848.200000   

segment             
feature     target  
timestamp           
2022-01-01   898.0  
2022-01-02  3167.0  
2022-01-03  3380.0  
2022-01-04  3423.0  
2022-01-05  3373.0

In [153]:
forecast.head()  # посмотрите на первые несколько строк прогноза


segment                         segment_1                                \
feature    date_flags_day_number_in_month date_flags_day_number_in_week   
timestamp                                                                 
2024-04-04                              4                             3   
2024-04-05                              5                             4   
2024-04-06                              6                             5   
2024-04-07                              7                             6   
2024-04-08                              8                             0   

segment                                                                       \
feature    date_flags_is_weekend date_flags_week_number_in_month fold_number   
timestamp                                                                      
2024-04-04                 False                               1           0   
2024-04-05                 False                               1           0   
2024-04-06                  True                               1           0   
2024-04-07                  True                               1           0   
2024-04-08                 False                               2           0   

segment                                                  
feature     lags_1  lags_2  lags_3    mean       target  
timestamp                                                
2024-04-04  2180.0  2045.0  2108.0  2451.0  2403.670747  
2024-04-05     NaN  2180.0  2045.0  2111.0  2664.210269  
2024-04-06     NaN     NaN  2180.0  2112.5  2767.492129  
2024-04-07     NaN     NaN     NaN  2180.0  2587.131828  
2024-04-08     NaN     NaN     NaN     0.0  1830.992950

In [154]:
# train_end = "2024-02-01"
# val_end = "2024-06-01"
# test_end = "2024-08-31"

# train_data = data[data["timestamp"] <= train_end]
# val_data = data[(data["timestamp"] > train_end) & (data["timestamp"] <= val_end)]
# test_data = data[(data["timestamp"] > val_end) & (data["timestamp"] <= test_end)]

# # Преобразуем в TSDataset
# train_tsd = TSDataset.to_dataset(train_data)
# train_ts = TSDataset(train_tsd, freq="D")

# val_tsd = TSDataset.to_dataset(val_data)
# val_ts = TSDataset(val_tsd, freq="D")

# test_tsd = TSDataset.to_dataset(test_data)
# test_ts = TSDataset(test_tsd, freq="D")


In [155]:
type(train_ts)

etna.datasets.tsdataset.TSDataset

In [156]:
# train_end = "2024-02-01"
# val_end = "2024-06-01"
# test_end = "2024-08-31"

# train_data = data[data["timestamp"] <= train_end]
# val_data = data[(data["timestamp"] > train_end) & (data["timestamp"] <= val_end)]
# test_data = data[(data["timestamp"] > val_end) & (data["timestamp"] <= test_end)]

# print(f"Train: {len(train_data)} rows")
# print(f"Validation: {len(val_data)} rows")
# print(f"Test: {len(test_data)} rows")